# Cleaning Dataset: Spotify Music Data

This notebook focuses on cleaning and preparing a Spotify dataset for machine learning and analysis. It aims to handle inconsistencies, fill missing values, and ensure that the data is suitable for subsequent modeling.

---

## Dataset Overview
The dataset contains the following columns:

- **track_id**: Unique identifier for tracks.
- **track_name**: Name of the track.
- **artist_name**: Name of the artist(s).
- **artist_count**: Number of artists for a track.
- **released_year**, **released_month**, **released_day**: Release date information.
- **in_spotify_playlists**, **in_spotify_charts**: Presence in Spotify playlists and charts.
- **streams**: Total number of streams.
- **in_apple_playlists**, **in_apple_charts**, **in_deezer_playlists**, **in_deezer_charts**, **in_shazam_charts**: Presence in Apple Music, Deezer, and Shazam charts.
- **bpm**, **key**, **mode**: Musical characteristics such as tempo, key, and mode (Major/Minor).
- **danceability**, **valence**, **energy**, **acousticness**, **instrumentalness**, **liveness**, **speechiness**: Audio features quantifying aspects like mood, energy, and instrumentalness.

### Data Characteristics

- **Data Types**: Includes numeric, categorical, and string data.
- **Distributions**: Numeric features like streams and BPM may require normalization or scaling for analysis.
- **Missing/Incorrect Data**: Some features, such as keys and release years, have missing or incorrect values.

In [1]:
# Importing the libraries
import pandas as pd

# Importing the dataset
df = pd.read_csv('dataset/spotify-2023.csv', encoding='ISO-8859-1')

# Checking for missing values
df.isnull().sum()

track_name               0
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64

## Data Cleaning Steps

In this section, we perform several data cleaning steps to ensure the dataset is ready for analysis. The steps include:

1. **Adding Track ID**: A unique identifier for each track is added to the dataframe.
2. **Removing Commas and Converting Columns to Numeric**: Columns such as `streams`, `in_shazam_charts`, and `in_deezer_playlists` are cleaned by removing commas and converting them to numeric types.
3. **Handling Missing Values**: Empty values in the `key` column are replaced with 0, and missing values in the `in_shazam_charts` column are filled with 0.
4. **Renaming Columns**: Columns are renamed for better readability, such as changing `artist(s)_name` to `artist_name` and removing percentage signs from audio feature columns.
5. **Adjusting Track Names**: Track names are cleaned by removing text within parentheses, brackets, and after hyphens.
6. **Fixing Corrupted Artist and Track Names**: Specific corrections are applied to artist and track names based on known issues.
7. **Correcting Release Year and Key Values**: Incorrect release years and key values are fixed, and missing key values are filled with default values.
8. **Correcting Metrics for Specific Tracks**: Audio feature metrics for specific tracks are corrected based on known issues.
9. **Filtering Out Invalid Tracks**: Tracks with known invalid data are removed from the dataset.
10. **Adjusting Artist and Track Names**: Artist and track names are standardized by converting to title case, stripping whitespace, and replacing multiple spaces with a single space.
11. **Modifying Existing Features**: The `key` column is converted to a categorical type, and the `artist_count` column is calculated based on the number of artists.
12. **Reordering Columns**: The columns are reordered to place `track_id` at the beginning.

These steps help to ensure that the dataset is clean, consistent, and ready for further analysis and modeling.

In [2]:
# Add track_id to DF
df['track_id'] = range(1, len(df) + 1)

# Remove commas and convert columns to numeric
for col in ['streams', 'in_shazam_charts', 'in_deezer_playlists']:
    df[col] = pd.to_numeric(df[col].str.replace(
        ",", "", regex=True), errors='coerce')

# Substitute empty values with 0
df['key'] = df['key'].replace("", 0).fillna(0)
df['in_shazam_charts'] = df['in_shazam_charts'].fillna(0)

# Rename columns for readability
df.rename(columns={
    'artist(s)_name': 'artist_name',
    'danceability_%': 'danceability',
    'valence_%': 'valence',
    'energy_%': 'energy',
    'acousticness_%': 'acousticness',
    'instrumentalness_%': 'instrumentalness',
    'liveness_%': 'liveness',
    'speechiness_%': 'speechiness',
}, inplace=True)

# Adjust track names
df['track_name'] = df['track_name'].str.replace(r'(?<!^)\(.*$', '', regex=True)
df['track_name'] = df['track_name'].str.replace(r'(?<!^)\[.*$', '', regex=True)
df['track_name'] = df['track_name'].str.replace(r'- .*', '', regex=True)

In [3]:
# Fix corrupted artist names
artist_name_corrections = {
    27: "Rema, Selena Gomez",
    44: "Tyler The Creator, Kali Uchis",
    64: "Rauw Alejandro, Rosalía",
    66: "Coldplay",
    113: "Jasiel Nuñez, Peso Pluma",
    119: "Sebastián Yatra, Manuel Turizo, Beéle",
    147: "Bomba Estéreo, Bad Bunny",
    205: "Arijit Singh, Sachin-Jigar, Amitabh Bhattacharya",
    210: "Kaliii",
    211: "Junior H, Eden Muñoz",
    213: "Semicenk, Doğu Swag",
    232: "Beyoncé",
    237: "Kendrick Lamar, Beyoncé",
    250: "Zé Neto & Cristiano",
    259: "Rich The Kid, Matuê",
    279: "Rosalía",
    295: "Jasiel Nuñez, Peso Pluma",
    303: "Tyler The Creator",
    304: "Tiësto, Tate McRae",
    305: "Tyler The Creator",
    310: "Marília Mendonça",
    376: "Wisin & Yandel, Rosalía",
    381: "Rosalía",
    393: "Rema",
    412: "Elley Duhé",
    446: "Michael Bublé",
    452: "José Feliciano",
    453: "Michael Bublé",
    489: "DJ LK da Escócia, Mc Jhenny, Tchakabum, Mc Ryan SP",
    504: "Lauren Spencer Smith",
    515: "The Weeknd, Tyler The Creator",
    527: "Måneskin",
    531: "Tiësto, Ava Max",
    534: "Olivia Rodrigo",
    537: "Xamã, Gustah, Neo Beats",
    544: "Justin Quiles, Lenny Tavárez, Blessd",
    574: "Tiësto, Karol G",
    576: "The Weeknd, Rosalía",
    599: "Tiësto",
    601: "Måneskin",
    602: "Marília Mendonça, George Henrique & Rodrigo",
    643: "Dimelo Flow, Sech, Lenny Tavárez, Arcangel, De La Ghetto, Justin Quiles, Dalex, Rich Music",
    646: "Maiara & Maraisa, Marília Mendonça",
    668: "La Pantera, Quevedo, Juseph, ABHIR, Bejo, Cruz Cafuné, EL IMA",
    669: "Dr. Dre, 2Pac, Roger Troutman",
    683: "Zé Felipe",
    689: "Marília Mendonça, Hugo & Guilherme",
    695: "Tyler The Creator",
    711: "Eden Muñoz",
    743: "Matuê, Teto & WIU",
    750: "Luísa Sonza, MC Frog, DJ Gabriel do Borel, Davi Kneip",
    760: "Jordan Fisher, Josh Levi, Finneas O'Connell, 4*TOWN, Topher Ngo, Grayson Villanueva",
    783: "Bad Bunny, The Marías",
    799: "Måneskin",
    834: "Pharrell Williams, Tyler The Creator, 21 Savage",
    864: "Beyoncé",
    870: "Schürze, DJ Robin",
    888: "Beyoncé",
    889: "Ghost B.C.",
    919: "Måneskin",
    930: "Luciano, Aitch, Bia"
}
df['artist_name'] = df['track_id'].map(
    artist_name_corrections).fillna(df['artist_name'])

# Fix corrupted track names
track_name_corrections = {
    37: "Frágil",
    61: "Tú",
    80: "CORAZÓN VA",
    83: "Novidade na Área",
    94: "Don't Blame Me",
    126: "Feliz Cumpleaños Fe",
    142: "Novo Balanço",
    175: "Idol",
    193: "Titi Me Preguntó",
    203: "Acróstico",
    213: "Pişman Değilim",
    235: "Niña Bonita",
    248: "Cartão B",
    259: "Conexões de Máfia",
    266: "Cupid - Twin Ver. (FIFTY FIFTY) - Sped Up Version",
    276: "Igualito a Mi Apá",
    279: "DESPECHÁ",
    302: "Arcángel: Bzrp Music Sessions, Vol 53",
    310: "Leão",
    337: "Ain't That Some",
    338: "Thinkin' Bout Me",
    347: "PLAYA DEL INGLÉS",
    352: "PERO TÚ",
    355: "Monotonía",
    356: "'98 braves",
    365: "Sem Aliança no Dedo",
    369: "LA CANCIÓN",
    370: "¿Qué Más Pues?",
    375: "Shinunoga E-Wa",
    380: "Devil Don't Cry",
    399: "Muñeca",
    440: "Agudo Mágico 3",
    489: "Tubarão Te Amo",
    500: "GATÚBELA",
    501: "abcdefu",
    515: "Here We Go... Again",
    537: "Malvadão 3",
    539: "Don't Break My Heart",
    548: "Volvó",
    558: "L'enfer",
    559: "Nostálgico",
    602: "Vai Lá Em Casa Hoje",
    619: "Cúrame",
    623: "¿Qué Más Pues?",
    645: "When I'm Gone",
    646: "Esqueça-me Se For Capaz",
    647: "Miénteme",
    648: "Súbele",
    668: "Cayó La Noche",
    680: "Problema",
    694: "Señorita",
    701: "Una Noche en Medellín",
    706: "DANÇA",
    723: "Money so Big",
    733: "X ÚLTIMA",
    746: "Me Arrepentí",
    749: "RUMBATÓN",
    767: "Después de la Playa",
    789: "Enséñame",
    790: "El Apagón",
    811: "TUS LÁGRIMAS",
    882: "cómo dormí",
    886: "Si Te La Encuentras Por Ahí",
    914: "XQ Te Pones Así",
    916: "Sin Señal"
}
df['track_name'] = df['track_id'].map(
    track_name_corrections).fillna(df['track_name'])

In [4]:
# Fix wrong release year
release_year_corrections = {
    641: 2017,
    911: 2011
}
df['released_year'] = df['track_id'].map(
    release_year_corrections).fillna(df['released_year'])

# Fill in missing key values
df['key'] = df['key'].replace(0, "C")
key_corrections = {125: "A", 460: "A#", 533: "G#"}
df['key'] = df['track_id'].map(key_corrections).fillna(df['key'])

# Correct metrics for specific track
metric_corrections = {
    'bpm': {534: 101},
    'danceability': {534: 38},
    'valence': {534: 8},
    'energy': {534: 34},
    'acousticness': {534: 69},
    'speechiness': {534: 3},
}
for metric, corrections in metric_corrections.items():
    df[metric] = df['track_id'].map(corrections).fillna(df[metric])

# Filter out tracks with wrong data
invalid_track_ids = [90, 124, 266, 394, 440, 575, 675, 813]
df = df[~df['track_id'].isin(invalid_track_ids)]

# Adjust track and artist names
df['artist_name'] = df['artist_name'].str.lower().str.title(
).str.strip().str.replace(r'\s+', ' ', regex=True)
df['track_name'] = df['track_name'].str.lower().str.title(
).str.strip().str.replace(r'\s+', ' ', regex=True)

# Modify existing features
df['key'] = pd.Categorical(df['key'], categories=[
                           "A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"])
df['artist_count'] = df['artist_name'].str.count(',') + 1

# Reorder columns
columns_order = ['track_id'] + [col for col in df.columns if col != 'track_id']
df = df[columns_order]

df

,track_id,track_name,artist_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,...,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
0,1,Seven,"Latto, Jung Kook",2,2023.0,7,14,553,147,141381703.0,...,125.0,B,Major,80.0,89.0,83.0,31.0,0,8,4.0
1,2,Lala,Myke Towers,1,2023.0,3,23,1474,48,133716286.0,...,92.0,C#,Major,71.0,61.0,74.0,7.0,0,10,4.0
2,3,Vampire,Olivia Rodrigo,1,2023.0,6,30,1397,113,140003974.0,...,138.0,F,Major,51.0,32.0,53.0,17.0,0,31,6.0
3,4,Cruel Summer,Taylor Swift,1,2019.0,8,23,7858,100,800840817.0,...,170.0,A,Major,55.0,58.0,72.0,11.0,0,11,15.0
4,5,Where She Goes,Bad Bunny,1,2023.0,5,18,3133,50,303236322.0,...,144.0,A,Minor,65.0,23.0,80.0,14.0,63,11,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,949,My Mind & Me,Selena Gomez,1,2022.0,11,3,953,0,91473363.0,...,144.0,A,Major,60.0,24.0,39.0,57.0,0,8,3.0
949,950,Bigger Than The Whole Sky,Taylor Swift,1,2022.0,10,21,1180,0,121871870.0,...,166.0,F#,Major,42.0,7.0,24.0,83.0,1,12,6.0
950,951,A Veces,"Feid, Paulo Londra",2,2022.0,11,3,573,0,73513683.0,...,92.0,C#,Major,80.0,81.0,67.0,4.0,0,8,6.0
951,952,En La De Ella,"Feid, Sech, Jhayco",3,2022.0,10,20,1320,0,133895612.0,...,97.0,C#,Major,82.0,67.0,77.0,8.0,0,12,5.0


In [5]:
# Save the cleaned dataset
df.to_csv('dataset/spotify-2023-cleaned.csv', index=False)

df.isnull().sum()

track_id                0
track_name              0
artist_name             0
artist_count            0
released_year           0
released_month          0
released_day            0
in_spotify_playlists    0
in_spotify_charts       0
streams                 0
in_apple_playlists      0
in_apple_charts         0
in_deezer_playlists     0
in_deezer_charts        0
in_shazam_charts        0
bpm                     0
key                     0
mode                    0
danceability            0
valence                 0
energy                  0
acousticness            0
instrumentalness        0
liveness                0
speechiness             0
dtype: int64